# Tutorial 1: FBA 기초

이 튜토리얼에서는 Simulator를 사용하여 기본적인 Flux Balance Analysis (FBA)를 수행합니다.

## 1. 환경 확인 및 라이브러리 임포트


In [ ]:
# 환경 확인
import sys
print(f"Python 경로: {sys.executable}")
print(f"Python 버전: {sys.version}")

# 필요한 라이브러리 임포트
from Simulator import Simulator
import cobra
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

print("\n✓ 모든 라이브러리 임포트 성공!")
print(f"COBRApy 버전: {cobra.__version__}")


## 2. 대사 모델 로드

E. coli iML1515 모델을 로드합니다.


In [ ]:
# Simulator 초기화
sim = Simulator()

# COBRApy에서 모델 로드
model = cobra.io.load_model("iML1515")

print(f"모델명: {model.id}")
print(f"반응 수: {len(model.reactions)}")
print(f"대사물질 수: {len(model.metabolites)}")
print(f"유전자 수: {len(model.genes)}")

# Simulator에 모델 로드
sim.load_cobra_model(model)
print("\n✓ 모델 로드 완료!")


## 3. 기본 FBA 실행

최적 성장 속도를 예측합니다.


In [ ]:
# FBA 실행
status, growth_rate, fluxes = sim.run_FBA()

print(f"=== FBA 결과 ===")
print(f"최적화 상태: {status}")
print(f"최적 성장 속도: {growth_rate:.4f} hr⁻¹")
print(f"활성 반응 수: {sum(1 for v in fluxes.values() if abs(v) > 1e-6)}")
print(f"전체 반응 수: {len(fluxes)}")


## 4. 주요 교환 반응 분석


In [ ]:
# 주요 교환 반응
exchange_reactions = {
    'EX_glc__D_e': 'Glucose 흡수',
    'EX_o2_e': 'Oxygen 흡수',
    'EX_co2_e': 'CO2 분비',
    'EX_ac_e': 'Acetate 분비',
    'EX_etoh_e': 'Ethanol 분비',
    'EX_for_e': 'Formate 분비'
}

print("=== 주요 교환 반응 ===")
exchange_data = []
for rxn_id, name in exchange_reactions.items():
    if rxn_id in fluxes:
        flux = fluxes[rxn_id]
        print(f"{name:20s}: {flux:8.4f} mmol/gDW/hr")
        exchange_data.append({'반응': name, '플럭스': flux})

# DataFrame으로 변환
df_exchange = pd.DataFrame(exchange_data)
df_exchange


## 5. 플럭스 분포 시각화

가장 높은 플럭스를 가진 반응들을 시각화합니다.


In [ ]:
# 상위 20개 플럭스 (절대값 기준)
top_fluxes = sorted(fluxes.items(), key=lambda x: abs(x[1]), reverse=True)[:20]
rxn_names, flux_values = zip(*top_fluxes)

# 시각화
plt.figure(figsize=(12, 8))
colors = ['#e74c3c' if v < 0 else '#3498db' for v in flux_values]
plt.barh(range(len(flux_values)), flux_values, color=colors)
plt.yticks(range(len(rxn_names)), rxn_names, fontsize=10)
plt.xlabel('Flux (mmol/gDW/hr)', fontsize=12)
plt.title('Top 20 Reactions by Absolute Flux', fontsize=14, fontweight='bold')
plt.axvline(x=0, color='black', linestyle='-', linewidth=0.5)
plt.grid(axis='x', alpha=0.3)
plt.tight_layout()
plt.show()


## 6. Objective 변경: 숙신산 생산 최대화

기본 objective는 biomass 성장이지만, 특정 대사물질의 생산을 최대화하도록 변경할 수 있습니다.


In [ ]:
# 기본 objective (biomass)로 FBA
print("=== 기본 Objective: Biomass 최대화 ===")
status, growth_rate, fluxes_biomass = sim.run_FBA()
succ_production_at_max_growth = fluxes_biomass.get('EX_succ_e', 0)
print(f"최대 성장 속도: {growth_rate:.4f} hr⁻¹")
print(f"숙신산 생산: {succ_production_at_max_growth:.4f} mmol/gDW/hr")

# Objective를 숙신산 생산으로 변경
print("\n=== Objective 변경: 숙신산 생산 최대화 ===")
status, max_succ, fluxes_succ = sim.run_FBA(
    new_objective='EX_succ_e',
    mode='max'
)

if status == 'optimal':
    growth_at_max_succ = fluxes_succ[sim.objective]  # 원래 objective (biomass)
    print(f"최대 숙신산 생산: {max_succ:.4f} mmol/gDW/hr")
    print(f"이때 성장 속도: {growth_at_max_succ:.4f} hr⁻¹")
    
    # 비교
    print(f"\n생산량 증가: {max_succ - succ_production_at_max_growth:.4f} mmol/gDW/hr")
    print(f"→ 성장을 희생하여 숙신산 생산을 {max_succ/max(succ_production_at_max_growth, 1e-6):.1f}배 증가")


## 7. Constraint 추가하여 FBA 실행

특정 반응에 flux constraint를 추가하여 대사 흐름을 제어할 수 있습니다.


In [ ]:
# 제약 조건 1: 최소 숙신산 생산 강제
print("=== 제약 조건 1: 최소 숙신산 생산 5 mmol/gDW/hr ===")
constraints_1 = {
    'EX_succ_e': (5.0, 1000)  # 최소 5, 최대 1000 (사실상 무제한)
}

status, growth_constrained, fluxes_constrained = sim.run_FBA(
    flux_constraints=constraints_1
)

if status == 'optimal':
    succ_prod = fluxes_constrained.get('EX_succ_e', 0)
    print(f"성장 속도: {growth_constrained:.4f} hr⁻¹")
    print(f"숙신산 생산: {succ_prod:.4f} mmol/gDW/hr")
    print(f"성장 감소: {(1 - growth_constrained/growth_rate)*100:.1f}%")
else:
    print("제약 조건을 만족하는 해가 없습니다!")

# 제약 조건 2: 산소 흡수 제한 (저산소 조건)
print("\n=== 제약 조건 2: 산소 제한 (최대 10 mmol/gDW/hr) ===")
constraints_2 = {
    'EX_o2_e': (-10.0, 0)  # 흡수는 음수, 최대 10까지만 흡수
}

status, growth_low_o2, fluxes_low_o2 = sim.run_FBA(
    flux_constraints=constraints_2
)

if status == 'optimal':
    o2_uptake = fluxes_low_o2.get('EX_o2_e', 0)
    print(f"성장 속도: {growth_low_o2:.4f} hr⁻¹")
    print(f"산소 흡수: {o2_uptake:.4f} mmol/gDW/hr")
    print(f"성장 감소: {(1 - growth_low_o2/growth_rate)*100:.1f}%")

# 제약 조건 3: 여러 제약 조건 동시 적용
print("\n=== 제약 조건 3: 산소 제한 + 최소 숙신산 생산 ===")
constraints_3 = {
    'EX_o2_e': (-10.0, 0),
    'EX_succ_e': (3.0, 1000)
}

status, growth_multi, fluxes_multi = sim.run_FBA(
    flux_constraints=constraints_3
)

if status == 'optimal':
    print(f"성장 속도: {growth_multi:.4f} hr⁻¹")
    print(f"산소 흡수: {fluxes_multi.get('EX_o2_e', 0):.4f} mmol/gDW/hr")
    print(f"숙신산 생산: {fluxes_multi.get('EX_succ_e', 0):.4f} mmol/gDW/hr")
else:
    print("제약 조건이 너무 엄격하여 해가 없습니다!")


## 8. MOMA 분석: Constraint와 Knockout

MOMA (Minimization of Metabolic Adjustment)는 유전자 knockout이나 환경 변화 후 대사 네트워크가 어떻게 적응하는지 예측합니다.


In [ ]:
# 야생형 참조 플럭스 (숙신산 생산을 강제한 상태)
print("=== 야생형: 최소 숙신산 생산 3 mmol/gDW/hr ===")
wt_constraints = {
    'EX_succ_e': (3.0, 1000)
}

_, wt_growth, wt_fluxes = sim.run_FBA(flux_constraints=wt_constraints)
wt_succ = wt_fluxes.get('EX_succ_e', 0)
print(f"야생형 성장: {wt_growth:.4f} hr⁻¹")
print(f"야생형 숙신산 생산: {wt_succ:.4f} mmol/gDW/hr")

# MOMA 분석 1: PGI knockout
print("\n=== MOMA 분석 1: PGI 반응 knockout ===")
knockout_constraints = {
    'PGI': (0, 0),  # PGI 반응 차단
    'EX_succ_e': (3.0, 1000)  # 여전히 최소 생산 유지 요구
}

status_moma, distance, moma_fluxes = sim.run_MOMA(
    wild_flux=wt_fluxes,
    flux_constraints=knockout_constraints
)

if status_moma == 'optimal':
    moma_growth = moma_fluxes[sim.objective]
    moma_succ = moma_fluxes.get('EX_succ_e', 0)
    
    print(f"MOMA 예측 성장: {moma_growth:.4f} hr⁻¹ ({(1-moma_growth/wt_growth)*100:.1f}% 감소)")
    print(f"MOMA 예측 숙신산 생산: {moma_succ:.4f} mmol/gDW/hr")
    print(f"대사 거리 (L1 norm): {distance:.4f}")
    
    # 가장 많이 변화된 반응들
    flux_changes = {
        rxn: abs(moma_fluxes[rxn] - wt_fluxes[rxn])
        for rxn in wt_fluxes.keys()
    }
    top_5_changes = sorted(flux_changes.items(), key=lambda x: x[1], reverse=True)[:5]
    
    print("\n가장 많이 변화된 반응 (Top 5):")
    for rxn, change in top_5_changes:
        if change > 1e-6:
            print(f"  {rxn}: {wt_fluxes[rxn]:.3f} → {moma_fluxes[rxn]:.3f} (Δ={change:.3f})")
else:
    print("MOMA 최적화 실패!")

# MOMA 분석 2: 산소 제한 + knockout
print("\n=== MOMA 분석 2: 산소 제한 + PFK knockout ===")
knockout_constraints_2 = {
    'PFK': (0, 0),  # PFK 반응 차단
    'EX_o2_e': (-15.0, 0),  # 산소 제한
    'EX_succ_e': (2.0, 1000)  # 최소 생산 감소
}

status_moma2, distance2, moma_fluxes2 = sim.run_MOMA(
    wild_flux=wt_fluxes,
    flux_constraints=knockout_constraints_2
)

if status_moma2 == 'optimal':
    moma2_growth = moma_fluxes2[sim.objective]
    moma2_succ = moma_fluxes2.get('EX_succ_e', 0)
    moma2_o2 = moma_fluxes2.get('EX_o2_e', 0)
    
    print(f"MOMA 예측 성장: {moma2_growth:.4f} hr⁻¹ ({(1-moma2_growth/wt_growth)*100:.1f}% 감소)")
    print(f"숙신산 생산: {moma2_succ:.4f} mmol/gDW/hr")
    print(f"산소 흡수: {moma2_o2:.4f} mmol/gDW/hr")
    print(f"대사 거리: {distance2:.4f}")
else:
    print("제약 조건이 너무 엄격하여 MOMA 해가 없습니다!")


## 9. Theoretical Yield 계산

Theoretical yield는 기질 1 몰당 생성할 수 있는 목표 생산물의 최대 몰수입니다.
숙신산의 이론적 수율을 계산해봅시다.


In [ ]:
# Theoretical Yield 계산
print("=== Theoretical Yield: 숙신산 (Succinate) ===\n")

# 1. 성장 없이 순수 생산만 최대화
print("1️⃣  성장 없이 숙신산 생산 최대화 (이론적 최대)")
no_growth_constraints = {
    sim.objective: (0, 0)  # Biomass 생산 차단
}

status_th, max_succ_theoretical, fluxes_th = sim.run_FBA(
    new_objective='EX_succ_e',
    mode='max',
    flux_constraints=no_growth_constraints
)

if status_th == 'optimal':
    glc_uptake_th = abs(fluxes_th.get('EX_glc__D_e', 10.0))  # Glucose 흡수 (음수)
    
    # Theoretical yield = 생산물 / 기질
    theoretical_yield = max_succ_theoretical / glc_uptake_th
    
    print(f"  최대 숙신산 생산: {max_succ_theoretical:.4f} mmol/gDW/hr")
    print(f"  Glucose 흡수: {glc_uptake_th:.4f} mmol/gDW/hr")
    print(f"  Theoretical Yield: {theoretical_yield:.4f} mol-succ/mol-glc")
    print(f"  이론적 최대: {theoretical_yield:.4f} (= {theoretical_yield*100:.1f}%)")

# 2. 최소 성장 유지하면서 생산 최대화
print("\n2️⃣  최소 성장 유지 (10%) + 숙신산 생산 최대화")
min_growth_constraints = {
    sim.objective: (0.1 * growth_rate, 1000)  # 최소 10% 성장 유지
}

status_mg, max_succ_with_growth, fluxes_mg = sim.run_FBA(
    new_objective='EX_succ_e',
    mode='max',
    flux_constraints=min_growth_constraints
)

if status_mg == 'optimal':
    glc_uptake_mg = abs(fluxes_mg.get('EX_glc__D_e', 10.0))
    growth_mg = fluxes_mg[sim.objective]
    
    practical_yield = max_succ_with_growth / glc_uptake_mg
    yield_efficiency = (practical_yield / theoretical_yield) * 100
    
    print(f"  숙신산 생산: {max_succ_with_growth:.4f} mmol/gDW/hr")
    print(f"  성장 속도: {growth_mg:.4f} hr⁻¹ ({(growth_mg/growth_rate)*100:.1f}% of max)")
    print(f"  Glucose 흡수: {glc_uptake_mg:.4f} mmol/gDW/hr")
    print(f"  Practical Yield: {practical_yield:.4f} mol-succ/mol-glc")
    print(f"  효율: {yield_efficiency:.1f}% of theoretical")

# 3. 수율 비교 표
print("\n3️⃣  수율 비교표")
comparison_data = []

# 조건 1: 최대 성장 (기본 FBA)
glc_max_growth = abs(fluxes_biomass.get('EX_glc__D_e', 10.0))
succ_max_growth = fluxes_biomass.get('EX_succ_e', 0)
yield_max_growth = succ_max_growth / glc_max_growth if glc_max_growth > 0 else 0

comparison_data.append({
    '조건': '최대 성장',
    '성장 (hr⁻¹)': f"{growth_rate:.4f}",
    '숙신산 (mmol/gDW/hr)': f"{succ_max_growth:.4f}",
    '수율 (mol/mol)': f"{yield_max_growth:.4f}",
    '이론치 대비 (%)': f"{(yield_max_growth/theoretical_yield)*100:.1f}"
})

# 조건 2: 최소 성장 + 최대 생산
comparison_data.append({
    '조건': '최소 성장 (10%)',
    '성장 (hr⁻¹)': f"{growth_mg:.4f}",
    '숙신산 (mmol/gDW/hr)': f"{max_succ_with_growth:.4f}",
    '수율 (mol/mol)': f"{practical_yield:.4f}",
    '이론치 대비 (%)': f"{yield_efficiency:.1f}"
})

# 조건 3: 이론적 최대
comparison_data.append({
    '조건': '이론적 최대 (무성장)',
    '성장 (hr⁻¹)': "0.0000",
    '숙신산 (mmol/gDW/hr)': f"{max_succ_theoretical:.4f}",
    '수율 (mol/mol)': f"{theoretical_yield:.4f}",
    '이론치 대비 (%)': "100.0"
})

df_yield = pd.DataFrame(comparison_data)
print(df_yield.to_string(index=False))

# 4. 탄소 수지 분석
print("\n4️⃣  탄소 수지 분석 (Glucose → Succinate)")
print(f"  Glucose (C6H12O6): 6 탄소")
print(f"  Succinate (C4H6O4): 4 탄소")
print(f"  이론적 최대 수율: 6/4 = 1.5 mol-succ/mol-glc")
print(f"  실제 계산된 수율: {theoretical_yield:.4f} mol-succ/mol-glc")
print(f"  탄소 효율: {(theoretical_yield/1.5)*100:.1f}% (CO2 손실 고려)")

print("\n✅ Theoretical yield 분석 완료!")


In [ ]:
# 성장 vs 생산 trade-off 시각화
growth_levels = np.linspace(0, 1.0, 11)  # 0% ~ 100% 성장
succ_production = []
yields = []

print("성장-생산 Trade-off 분석 중...")
for fraction in growth_levels:
    min_growth = fraction * growth_rate
    
    constraints = {
        sim.objective: (min_growth, 1000)
    }
    
    status, max_prod, fluxes_temp = sim.run_FBA(
        new_objective='EX_succ_e',
        mode='max',
        flux_constraints=constraints
    )
    
    if status == 'optimal':
        glc = abs(fluxes_temp.get('EX_glc__D_e', 10.0))
        succ_production.append(max_prod)
        yields.append(max_prod / glc if glc > 0 else 0)
    else:
        succ_production.append(0)
        yields.append(0)

# 2x1 subplot 생성
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# 첫 번째 그래프: 성장 vs 생산
ax1.plot(growth_levels * growth_rate, succ_production, 'o-', 
         color='#e74c3c', linewidth=2, markersize=8)
ax1.axhline(y=theoretical_yield * 10, color='gray', linestyle='--', 
            label=f'Theoretical max (no growth)')
ax1.set_xlabel('Growth Rate (hr⁻¹)', fontsize=12)
ax1.set_ylabel('Succinate Production (mmol/gDW/hr)', fontsize=12)
ax1.set_title('Growth vs Production Trade-off', fontsize=14, fontweight='bold')
ax1.grid(alpha=0.3)
ax1.legend()

# 두 번째 그래프: 성장 vs 수율
ax2.plot(growth_levels * growth_rate, yields, 's-', 
         color='#3498db', linewidth=2, markersize=8)
ax2.axhline(y=theoretical_yield, color='gray', linestyle='--', 
            label=f'Theoretical yield: {theoretical_yield:.3f}')
ax2.set_xlabel('Growth Rate (hr⁻¹)', fontsize=12)
ax2.set_ylabel('Yield (mol-succ/mol-glc)', fontsize=12)
ax2.set_title('Growth vs Yield', fontsize=14, fontweight='bold')
ax2.grid(alpha=0.3)
ax2.legend()

plt.tight_layout()
plt.show()

print(f"\n✅ 성장과 생산의 Trade-off:")
print(f"  - 최대 성장 시: {succ_production[10]:.4f} mmol/gDW/hr (수율: {yields[10]:.4f})")
print(f"  - 무성장 시: {succ_production[0]:.4f} mmol/gDW/hr (수율: {yields[0]:.4f})")
print(f"  - 최적 균형점: 성장 {growth_levels[5]*100:.0f}% → 생산 {succ_production[5]:.4f} mmol/gDW/hr")


## 11. 요약

이 튜토리얼에서는 Simulator의 다양한 기능을 실습했습니다:

### 기본 분석
- ✅ FBA를 사용한 최적 성장 속도 예측
- ✅ 주요 교환 반응 분석
- ✅ 플럭스 분포 시각화

### 고급 분석
- ✅ **Objective 변경**: 숙신산 생산 최대화
- ✅ **Constraint 추가**: 산소 제한, 최소 생산 강제 등
- ✅ **MOMA 분석**: Knockout과 환경 변화에 대한 대사 적응 예측
- ✅ **Theoretical Yield**: 이론적 최대 수율 계산 및 비교
- ✅ **Trade-off 분석**: 성장과 생산 간의 균형 시각화

### 주요 학습 내용
1. **Objective 변경**: `new_objective` 파라미터로 목적 함수 변경
2. **Constraint 설정**: `flux_constraints` 딕셔너리로 반응 제약
3. **MOMA**: 야생형 참조 플럭스 대비 최소 변화 예측
4. **Yield 분석**: 기질 대비 생산물의 효율 계산

### 다음 단계
- pFBA (Parsimonious FBA)로 최소 플럭스 분포 찾기
- ROOM으로 knockout 표현형 예측
- FSEOF/FVSEOF로 생산 envelope 분석
- 다양한 대사공학 전략 설계 및 평가
